In [ ]:
import numpy as np
import pandas as pd
import itertools
import glob
from openpyxl import load_workbook
Auftrag = './absys_files/A-2017-0059_919A.XLSX'

In [ ]:
def processZaehlernummer(Zaehlernummer):
    if isinstance(int(Zaehlernummer), int):
        assert len(str(Zaehlernummer)) == 8
        print(Zaehlernummer)

In [ ]:
def protoCleanup():
    df2append = df.dropna()
    df2append = df2append[(df2append['Baulänge'] != 'Baulänge') &
                          (df2append['Bauhöhe'] != 'Bauhöhe')&
                          (df2append['Bautiefe'] != 'Bautiefe') &
                          (df2append['ZählernummerAlt'] != 'Altgerät') &
                          (df2append['ZählernummerNeu'] != 'Neugerät')]
    df2append = df2append[(df2append['Baulänge'] != 'D') &
                          (df2append['Bauhöhe'] != 'E') &
                          (df2append['Bautiefe'] != 'F') &
                          (df2append['ZählernummerAlt'] != 'Nr.') &
                          (df2append['ZählernummerNeu'] != 'Nr.')]

In [ ]:
def collectProtocols(pathPattern='./absys_files/SenftenbergerRing*'):
    protoColumns = {'Baulänge':str,
                    'Bauhöhe':str,
                    'Bautiefe':str,
                    'ZählernummerAlt':str,
                    'ZählernummerNeu':str}
    protokolle = glob.glob(pathPattern)
    #ToDo: Fehlerbehandlung wenn keine Dateien gefunden werden.
    protoDF = pd.DataFrame(columns=protoColumns.keys())
    for protokol in protokolle:
        protoWB = load_workbook(protokol)
        #Es wird vorrausgesetzt, dass das Worksheet "Dokumentation" heißt.
        try:
            protoWS = protoWB['Dokumentation']
        except:
            print("In der Datei "+protokol+" wurde kein Worksheet 'Dokumentation' gefunden!")
            break
        df = pd.read_excel(protokol, 
                           sheetname=0, 
                           names=protoColumns.keys(), 
                           dtype=protoColumns,  
                           parse_cols="E:G,L,V")
        df2append = protoCleanup(df)
        protoDF = protoDF.append(df2append)
    return protoDF

In [ ]:
#Nimmt an, dass das Excel-sheet IMMER 'Zählerliste' heißt. Tippfehler gehen schief!!!
df = pd.read_excel(Auftrag, sheetname='Zählerliste', header=9)
protoDF = collectProtocols()
#Es wird vorrausgesetzt, das die Zaehlernummern immer in der 5.(E) Spalte stehen.
for Zaehlernummer in df['alte\nZähler-Nr']:
    #Fehlerbehandlung, wenn Feldinhalt keine Zählernummer
    try:
        processZaehlernummer(Zaehlernummer)
    except:
        print('Bei der Bearbeitung von Zaehlernummer '+Zaehlernummer+' ist ein Fehler aufgetreten')
df